In [2]:
# 1. 导入模块
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 2. 加载 .env 环境变量
load_dotenv()

# 3. 配置 API Key
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

if not API_KEY:
    raise ValueError("未检测到 API_KEY，请检查 .env 文件是否配置正确")

# 4. 初始化大模型
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3
)

# 5. 构造 Prompt（教学阶段用字符串更直观）
prompt = "请写一段50字左右的 AI 学习建议，语言简洁、实用，适合初学者。"

# 6. 调用模型
response = llm.invoke(prompt)

# 7. 输出结果
print("生成的学习建议：")
print(response.content)


生成的学习建议：
1. 先动手：从简单项目开始，跑通代码比纯理论更重要。
2. 重基础：掌握Python、线性代数和概率论核心概念。
3. 读代码：多分析经典开源项目，理解实现逻辑。
4. 常复现：尝试复现论文基础模型，锻炼工程能力。
5. 勤交流：加入社区，提问和分享能突破学习瓶颈。

坚持小步快跑，保持每周实践，半年可见成效。


In [3]:
# 1. 导入需要的模块
import os 
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv

# 2. 加载API密钥
load_dotenv()

# 3. 初始化大模型（和LangChain案例一样）
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"

if not API_KEY:
    raise ValueError("未检测到 API_KEY，请检查 .env 文件是否配置正确")

# 4. 初始化大模型
llm = ChatOpenAI(
    api_key=API_KEY,
    base_url=BASE_URL,
    model="deepseek-chat",
    temperature=0.3
)

# 4. 定义 State
class WorkflowState(TypedDict):
    user_role: str
    original_advice: str
    simplified_advice: str

# 5. 定义节点
def generate_advice(state: WorkflowState):
    prompt = f"给{state['user_role']}写一段50字左右的 AI 学习建议。"
    result = llm.invoke(prompt)
    return {"original_advice": result.content}

def simplify_advice(state: WorkflowState):
    prompt = f"把下面的学习建议精简到30字以内：{state['original_advice']}"
    result = llm.invoke(prompt)
    return {"simplified_advice": result.content}

# 6. 构建工作流
workflow = StateGraph(WorkflowState)

workflow.add_node("generate", generate_advice)
workflow.add_node("simplify", simplify_advice)

workflow.add_edge(START, "generate")
workflow.add_edge("generate", "simplify")
workflow.add_edge("simplify", END)

app = workflow.compile()

# 7. 执行
result = app.invoke({"user_role": "高校学生"})

# 8. 输出
print("原始学习建议：")
print(result["original_advice"])
print("\n精简后学习建议：")
print(result["simplified_advice"])

原始学习建议：
AI时代，学习建议：掌握基础数学与编程，保持批判思维；动手实践项目，关注伦理问题；善用AI工具辅助学习，但不忘深度思考。保持好奇，持续探索。

精简后学习建议：
学基础，练批判，做项目，善用AI，勤思考，持好奇。
